<a href="https://colab.research.google.com/github/Rakesh-SSN/FYP/blob/master/BERT_TASK_2_PUNJABI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [141]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')


import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")



!pip install transformers

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [142]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [143]:
import pandas as pd

# Load the dataset into a pandas dataframe.
df = pd.read_csv("/content/drive/My Drive/FYP/bert/glue/cola/punjabi/task2punjabi.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 2,200



,sentence_source,label,label_notes,sentence
1451,PUN1452,0,a,ਮੋਬਾਈਲ ਇੰਟਰਨੈੱਟ ਸੇਵਾਵਾਂ ਲਗਾਤਾਰ ਬੰਦ ਹਨ। ਵੱਖਵਾਦੀ...
1334,PUN1335,0,a,ਸਵੇਰ ਦਾ ਪੇਪਰ 9.30 ਵਜੇ ਸ਼ੁਰੂ ਹੋਇਆ ਸੀ ਅਤੇ 11.30 ...
1761,PUN1762,0,a,ਵਿਜੈ ਰੁਪਾਨੀ ਹੋਣਗੇ ਗੁਜਰਾਤ ਦੇ ਮੁੱਖ ਮੰਤਰੀ<eol>ਦਿੱ...
1735,PUN1736,0,a,"ਸ਼੍ਰੋਮਣੀ ਅਕਾਲੀ ਦਲ (ਬ) ਵੱਲੋਂ ਐਨ ਆਰ ਆਈ ਵਿੰਗ, ਇਟਲ..."
1576,PUN1577,0,a,ਚਿੱਤਰਕਾਰ ਐਸ.ਐਚ. ਰਜ਼ਾ ਦਾ ਦਿਹਾਂਤ<eol>ਚੰਡੀਗੜ੍ਹ ਯੂ...
1110,PUN1111,2,a,ਮੌਕੇ ਤੋਂ ਇਕੱਤਰ ਕੀਤੀ ਜਾਣਕਾਰੀ ਅਨੁਸਾਰ ਜ਼ੀਰਕਪੁਰ-ਪਟਿ...
1594,PUN1595,0,a,ਬੀ.ਐਸ.ਐਨ.ਐਲ. ਦੇ ਡਾਟਾ ਪਲੈਨ ਹੁਣ ਜ਼ਰੂਰਤ ਮੁਤਾਬਿਕ<e...
530,PUN0531,1,a,ਸ੍ਰੀ ਕੇਜਰੀਵਾਲ ਪੰਜਾਬ ਦਾ ਮੁੱਖ ਮੰਤਰੀ ਬਣਨ ਤੋਂ ਬਾਅਦ...
651,PUN0652,1,a,ਅਧਿਕਾਰੀਆਂ ਨੇ ਕਿਹਾ ਕਿ ਵਿਜੈ ਮਾਲਿਆ ਅਤੇ ਉਸ ਦੇ ਪਰਿਵ...
819,PUN0820,2,a,ਬਠਿੰਡਾ-ਬਾਦਲ ਸੜਕ ਮਾਰਗ ’ਤੇ ਵੱਡੇ ਭਾਰੀ ਦਰਖ਼ਤ ਵੀ ਕੁਹ...


In [144]:
df.loc[df.label == 0].sample(5)[['sentence', 'label']]
print(df.label)

0       1
1       1
2       1
3       1
4       1
       ..
2195    0
2196    0
2197    0
2198    0
2199    0
Name: label, Length: 2200, dtype: int64


In [0]:
sentences = df.sentence.values
labels = df.label.values



In [146]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=True)

Loading BERT tokenizer...


In [147]:
# Print the original sentence.
print(' Original: ', sentences[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  ਰੀਓ ਉਲੰਪਿਕ ਜਾਵੇਗਾ ਨਰਸਿੰਘ<eol>ਨਰਸਿੰਘ ਜਾਵੇਗਾ ਰੀਓ ਉਲਪਿੰਕ
Tokenized:  ['ਰ', '##ੀ', '##ਓ', 'ਉ', '##ਲ', '##ਪ', '##ਿਕ', 'ਜਾ', '##ਵ', '##ਗ', '##ਾ', 'ਨ', '##ਰ', '##ਸ', '##ਿ', '##ਘ', '<', 'eo', '##l', '>', 'ਨ', '##ਰ', '##ਸ', '##ਿ', '##ਘ', 'ਜਾ', '##ਵ', '##ਗ', '##ਾ', 'ਰ', '##ੀ', '##ਓ', 'ਉ', '##ਲ', '##ਪ', '##ਿਕ']
Token IDs:  [1041, 13094, 111248, 1011, 15423, 31948, 26674, 62130, 32143, 48874, 13768, 1034, 13475, 19836, 45378, 111250, 133, 13934, 10161, 135, 1034, 13475, 19836, 45378, 111250, 62130, 32143, 48874, 13768, 1041, 13094, 111248, 1011, 15423, 31948, 26674]


In [148]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'

                        # This function also supports truncation and conversion
                        # to pytorch tensors, but we need to do padding, so we
                        # can't use these features :( .
                        #max_length = 128,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  ਰੀਓ ਉਲੰਪਿਕ ਜਾਵੇਗਾ ਨਰਸਿੰਘ<eol>ਨਰਸਿੰਘ ਜਾਵੇਗਾ ਰੀਓ ਉਲਪਿੰਕ
Token IDs: [101, 1041, 13094, 111248, 1011, 15423, 31948, 26674, 62130, 32143, 48874, 13768, 1034, 13475, 19836, 45378, 111250, 133, 13934, 10161, 135, 1034, 13475, 19836, 45378, 111250, 62130, 32143, 48874, 13768, 1041, 13094, 111248, 1011, 15423, 31948, 26674, 102]
Original:  ਰੀਓ ਉਲੰਪਿਕ ਜਾਵੇਗਾ ਨਰਸਿੰਘ<eol>ਨਰਸਿੰਘ ਜਾਵੇਗਾ ਰੀਓ ਉਲਪਿੰਕ
Token IDs: [101, 1041, 13094, 111248, 1011, 15423, 31948, 26674, 62130, 32143, 48874, 13768, 1034, 13475, 19836, 45378, 111250, 133, 13934, 10161, 135, 1034, 13475, 19836, 45378, 111250, 62130, 32143, 48874, 13768, 1041, 13094, 111248, 1011, 15423, 31948, 26674, 102]


In [149]:
print('Max sentence length: ', max([len(sen) for sen in input_ids]))

Max sentence length:  283
Max sentence length:  283


In [150]:
# We'll borrow the `pad_sequences` utility function to do this.
from keras.preprocessing.sequence import pad_sequences

# Set the maximum sequence length.
# I've chosen 64 somewhat arbitrarily. It's slightly larger than the
# maximum training sentence length of 47...
MAX_LEN = 64

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# Pad our input tokens with value 0.
# "post" indicates that we want to pad and truncate at the end of the sequence,
# as opposed to the beginning.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

print('\nDone.')


Padding/truncating all sentences to 64 values...

Padding token: "[PAD]", ID: 0

Done.

Padding/truncating all sentences to 64 values...

Padding token: "[PAD]", ID: 0

Done.


In [0]:
# Create attention masks
attention_masks = []

# For each sentence...
for sent in input_ids:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

In [152]:
# Use train_test_split to split our data into train and validation sets for
# training
from sklearn.model_selection import train_test_split

# Use 90% for training and 20% for validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.2)
# print(train_inputs)
# print(train_labels)

# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2018, test_size=0.2)

print(labels)
#print(train_masks)

[1 1 1 ... 0 0 0]
[1 1 1 ... 0 0 0]


In [0]:
# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.


train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [0]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.

batch_size = 32

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [155]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 3, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [156]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (119547, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              

In [0]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


In [0]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [0]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [131]:
import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch    40  of     55.    Elapsed: 0:00:17.

  Average training loss: 0.88
  Training epcoh took: 0:00:23

Running Validation...
  Accuracy: 0.68
  Validation took: 0:00:02

======== Epoch 2 / 4 ========
Training...
  Batch    40  of     55.    Elapsed: 0:00:17.

  Average training loss: 0.66
  Training epcoh took: 0:00:24

Running Validation...
  Accuracy: 0.68
  Validation took: 0:00:02

======== Epoch 3 / 4 ========
Training...
  Batch    40  of     55.    Elapsed: 0:00:17.

  Average training loss: 0.60
  Training epcoh took: 0:00:23

Running Validation...
  Accuracy: 0.68
  Validation took: 0:00:02

======== Epoch 4 / 4 ========
Training...
  Batch    40  of     55.    Elapsed: 0:00:17.

  Average training loss: 0.54
  Training epcoh took: 0:00:23

Running Validation...
  Accuracy: 0.68
  Validation took: 0:00:02

Training complete!

======== Epoch 1 / 4 ========
Training...
  Batch    40  of     55.    Elapsed: 0:00:17.

  Average tra

In [132]:
import pandas as pd

# Load the dataset into a pandas dataframe.
df = pd.read_csv("/content/drive/My Drive/FYP/bert/glue/cola/testdata/task2punjabi-test.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(df.shape[0]))

# Create sentence and label lists
sentences = df.sentence.values
labels = df.label.values

#print(len(labels))
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                   )
    
    input_ids.append(encoded_sent)

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)

#print(len(prediction_labels))
# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Number of test sentences: 750



In [133]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch

  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]
  
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  #print(outputs)
  # print(logits)
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)
  # # print(predictions)
  #print(true_labels)
print('    DONE.')

#print(true_labels)
print("*************************")

print(len(predictions))
#print(outputs)





Predicting labels for 750 test sentences...
    DONE.
*************************
24


In [134]:

count_NP=0
count_P=0
count_line=1
x=-1
true_count,false_count=0,0
print("label \t sno\tlogit\t\t\t\t\tindex")
for i in range(len(predictions)):
  for j in range(len(predictions[i])):
    print("(",end="")
    print(true_labels[i][j],end="")
    print(")",end="")
    print("\t",count_line,end="")
    # print("-  ",end="")
    if(predictions[i][j][0]>predictions[i][j][1] and predictions[i][j][0]>predictions[i][j][2]):
      #count_P+=1 
      #print(" Non Paraphrase         ",end="")
      print("\t",predictions[i][j],"\t0\t",end="")
      x=0
    elif(predictions[i][j][1]>predictions[i][j][0] and predictions[i][j][1]>predictions[i][j][2]):
      #print(" Paraphrase     ",end="")
     # count_NP+=1      
      print("\t",predictions[i][j],"\t1\t",end="")
      x=1
    elif(predictions[i][j][2]>predictions[i][j][0] and predictions[i][j][2]>predictions[i][j][1]):
      #print(" Semi Paraphrase     ",end="")
      #count_NP+=1      
      print("\t",predictions[i][j],"\t2\t",end="")
      x=2
    count_line+=1
    #print("\t",predictions[i][j],"\t2\t",end="")

    if(true_labels[i][j]==x):
      true_count+=1
      print("true")
    else:
      print("false")
      false_count+=1

print("Number of true predictions:",true_count)
print("Number of false predictions:",false_count)
  


label 	 sno	logit					index
(0)	 1	 [ 0.39383778 -0.8610478   0.5250055 ] 	2	false
(1)	 2	 [ 0.39383778 -0.8610478   0.5250055 ] 	2	false
(0)	 3	 [ 2.6725204 -1.6996533 -0.8029047] 	0	true
(1)	 4	 [-2.4261615   2.9653232  -0.88414335] 	1	true
(0)	 5	 [ 1.5865573 -1.6643666 -0.0906079] 	0	true
(1)	 6	 [-2.167908    0.47147217  1.2999414 ] 	2	false
(0)	 7	 [ 2.6217906 -1.5122868 -0.9503421] 	0	true
(1)	 8	 [-2.13611    3.0636902 -1.2166771] 	1	true
(0)	 9	 [ 1.9656068  -1.7430431  -0.10993354] 	0	true
(1)	 10	 [-1.7736821   0.00329567  1.5185819 ] 	2	false
(0)	 11	 [ 1.6448426  -1.6411881   0.11821485] 	0	true
(1)	 12	 [-2.541482   1.477618   0.5995183] 	1	true
(0)	 13	 [ 2.4763062  -1.6923548  -0.56689847] 	0	true
(1)	 14	 [-2.3023934  1.6217672  0.094087 ] 	1	true
(2)	 15	 [-1.2300369  -0.28488177  1.1128881 ] 	2	true
(0)	 16	 [ 2.1268482  -1.7996606  -0.13250239] 	0	true
(1)	 17	 [-1.8151836   0.03271659  1.4978063 ] 	2	false
(2)	 18	 [-1.3451625 -0.505891   1.6243416] 	2	true
(0)	 19

In [135]:
print("Accuracy:",true_count/count_line*100,"%")

Accuracy: 69.77363515312916 %


In [136]:

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

scores = [3.0, 1.0, 0.2]
for i in range(len(predictions)):
  for j in range(len(predictions[i])):
    predictions[i][j]=softmax(predictions[i][j])
    print(predictions[i][j])


[0.41232565 0.11755753 0.4701168 ]
[0.41232565 0.11755753 0.4701168 ]
[0.9582469  0.01209669 0.02965647]
[0.00444044 0.9748048  0.02075466]
[0.81589925 0.03160664 0.15249409]
[0.02124395 0.29751134 0.68124473]
[0.9577505  0.0153407  0.02690883]
[0.0054129  0.9810122  0.01357483]
[0.8695658  0.02131357 0.1091206 ]
[0.02957209 0.1748291  0.79559886]
[0.7970228  0.02981026 0.17316686]
[0.01253478 0.6975733  0.28989184]
[0.94059676 0.01455382 0.04484938]
[0.01597571 0.8085399  0.17548434]
[0.0715059  0.18399988 0.7444942 ]
[0.8895764  0.01753566 0.09288803]
[0.0287244  0.18229882 0.7889768 ]
[0.04386554 0.10153471 0.8545998 ]
[0.9536214  0.01224842 0.03413024]
[0.03356393 0.29363468 0.67280143]
[0.41646174 0.17522761 0.40831065]
[0.32532427 0.1261119  0.5485639 ]
[0.8448213  0.02057385 0.13460487]
[0.02884135 0.16134006 0.8098186 ]
[0.38087887 0.17821254 0.4409086 ]
[0.38087887 0.17821254 0.4409086 ]
[0.40902048 0.17682682 0.41415274]
[0.40902048 0.17682682 0.41415274]
[0.959073   0.015261

In [0]:
import numpy as np
np.savetxt("/content/drive/My Drive/FYP/bert/glue/cola/task2punjabi-results.txt",predictions, fmt="%s")



In [138]:
from sklearn.metrics import matthews_corrcoef

matthews_set = []

# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')

# For each input batch...
for i in range(len(true_labels)):
  
  # The predictions for this batch are a 2-column ndarray (one column for "0" 
  # and one column for "1"). Pick the label with the highest value and turn this
  # in to a list of 0s and 1s.
  pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
  
  

  # Calculate and store the coef for this batch.  
  matthews = matthews_corrcoef(true_labels[i], pred_labels_i)
                 
  matthews_set.append(matthews)
  
 # print(pred_labels_i)



Calculating Matthews Corr. Coef. for each batch...


In [139]:
matthews_set
# Combine the predictions for each batch into a single list of 0s and 1s.
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = [item for sublist in true_labels for item in sublist]

# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('MCC: %.3f' % mcc)

MCC: 0.555


In [140]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = '/content/drive/My Drive/FYP/bert/model-colab-task2malayalam'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))


Saving model to /content/drive/My Drive/FYP/bert/model-colab-task2malayalam


('/content/drive/My Drive/FYP/bert/model-colab-task2malayalam/vocab.txt',
 '/content/drive/My Drive/FYP/bert/model-colab-task2malayalam/special_tokens_map.json',
 '/content/drive/My Drive/FYP/bert/model-colab-task2malayalam/added_tokens.json')